In [1]:
from nltk.tokenize import sent_tokenize

In [2]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.stats import mode

2024-03-29 17:42:15.949648: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 17:42:15.949702: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 17:42:15.951161: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-29 17:42:15.957770: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Assuming all models have been correctly saved with their configurations and tokenizer
# For RoBERTa V3 and V4 (based on `roberta-small`)
tokenizer_small = AutoTokenizer.from_pretrained('Roberta_V3_1_task12/tokenizer')

# For RoBERTa V6 (based on `roberta-large`)
tokenizer_large = AutoTokenizer.from_pretrained('Roberta_V7_task12/tokenizer')

# Load models
model_v2 = TFAutoModelForSequenceClassification.from_pretrained('Roberta_V3_task12/model')
model_v3 = TFAutoModelForSequenceClassification.from_pretrained('Roberta_V3_1_task12/model')
model_v4 = TFAutoModelForSequenceClassification.from_pretrained('Roberta_V3_1p_task12/model')
model_v6 = TFAutoModelForSequenceClassification.from_pretrained('Roberta_V7_task12/model')

2024-03-29 17:42:19.946010: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-03-29 17:42:19.946343: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21222 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at Roberta_V3_task12/model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceCla

In [4]:
# Expanded contraction mapping
contractions_dict = {
    "i'm": "i am", "im": "i am", "i m": "i am",
    "you're": "you are", "youre": "you are",
    "he's": "he is", "hes": "he is",
    "she's": "she is", "shes": "she is",
    "it's": "it is", "its": "it is",
    "we're": "we are", "were": "we are",
    "they're": "they are", "theyre": "they are",
    "that's": "that is", "thats": "that is", "that s": "that is",
    "that'd": "that would", "thatd": "that would",
    "who's": "who is", "whos": "who is",
    "what's": "what is", "whats": "what is",
    "where's": "where is", "wheres": "where is",
    "when's": "when is", "whens": "when is",
    "why's": "why is", "whys": "why is",
    "how's": "how is", "hows": "how is",
    "ain't": "am not", "aint": "am not",
    "aren't": "are not", "arent": "are not",
    "isn't": "is not", "isnt": "is not",
    "wasn't": "was not", "wasnt": "was not",
    "weren't": "were not", "werent": "were not",
    "haven't": "have not", "havent": "have not",
    "hasn't": "has not", "hasnt": "has not",
    "hadn't": "had not", "hadnt": "had not",
    "won't": "will not", "wont": "will not",
    "wouldn't": "would not", "wouldnt": "would not",
    "don't": "do not", "dont": "do not",
    "doesn't": "does not", "doesnt": "does not",
    "didn't": "did not", "didnt": "did not",
    "can't": "cannot", "cant": "cannot",
    "couldn't": "could not", "couldnt": "could not",
    "shouldn't": "should not", "shouldnt": "should not",
    "mightn't": "might not", "mightnt": "might not",
    "mustn't": "must not", "mustnt": "must not",
}

# Create a regex pattern for contractions, enabling case-insensitive matching
contraction_patterns = r'\b(' + '|'.join([re.escape(k) for k in contractions_dict.keys()]) + r')\b'
contractions_re = re.compile(contraction_patterns, re.IGNORECASE)

In [5]:
def expand_contractions(text, contractions_dict=contractions_dict):
    def replace(match):
        # Fetch the correct replacement string from the dictionary using lowercase match
        return contractions_dict[match.group().lower()]
    # Use the sub method to replace all matches
    expanded_text = contractions_re.sub(replace, text)
    return expanded_text

def clean_text(text):
    text = text.strip()  # Strip leading/trailing whitespace
    text = expand_contractions(text)  # Expand contractions
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)  # Remove emoticons and unicode characters
    
    # Split text into sentences, strip each sentence, then join back with a space
    sentences = sent_tokenize(text)
    cleaned_sentences = [sentence.strip() for sentence in sentences]
    text = ' '.join(cleaned_sentences)
    
    return text

In [6]:
# Load the test dataset
test_df = pd.read_csv('test (1).csv', sep='\t')
test_df['sentence'] = test_df['sentence'].apply(clean_text)
def prepare_data(tokenizer, sentences, max_length=128):
    input_ids = []
    attention_masks = []

    for sent in sentences:
        encoded_dict = tokenizer.encode_plus(
            sent,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',  # PyTorch tensors
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert lists into tensors
    input_ids = tf.concat(input_ids, axis=0)
    attention_masks = tf.concat(attention_masks, axis=0)
    return input_ids, attention_masks

# Prepare data with both tokenizers
test_input_ids_small, test_attention_masks_small = prepare_data(tokenizer_small, test_df['sentence'].values)


In [7]:
def softmax(logits):
    if logits.ndim == 1:
        # This means we have only one set of logits, we expand the dimension to apply softmax
        logits = np.expand_dims(logits, 0)
    e_logits = np.exp(logits - np.max(logits, axis=-1, keepdims=True))
    return e_logits / e_logits.sum(axis=-1, keepdims=True)

def get_average_confidence_score(predictions):
    # First, ensure predictions is a 2D array
    if predictions.ndim == 1:
        predictions = np.expand_dims(predictions, 0)
        
    probabilities = softmax(predictions)
    average_confidence_per_class = np.mean(probabilities, axis=0)
    return average_confidence_per_class

In [ ]:
def get_predictions_tf(model, input_ids, attention_masks):
    # Use the model to predict the outcomes
    outputs = model.predict([input_ids, attention_masks])
    # Access the logits from the structured output
    logits = outputs.logits
    # Use np.argmax to get the most likely class label
    predicted_labels = np.argmax(logits, axis=1)
    return predicted_labels
predictions_v2 = get_predictions_tf(model_v2, test_input_ids_small, test_attention_masks_small)
predictions_v3 = get_predictions_tf(model_v3, test_input_ids_small, test_attention_masks_small)
predictions_v4 = get_predictions_tf(model_v4, test_input_ids_small, test_attention_masks_small)
predictions_v6 = get_predictions_tf(model_v6, test_input_ids_small, test_attention_masks_small)


45/45 [==============================] - 8s 88ms/step


In [ ]:
def reassign_classes_with_count(predictions_stack, threshold_ratio):
    happiness_index = 3  # index of 'happiness' in emotion_mapping
    surprise_index = 5  # index of 'surprise' in emotion_mapping
    change_count = 0  # Initialize the count of changed sentences
    
    # Get the indices of the maximum confidence scores along axis=0
    max_confidence_indices = np.argmax(predictions_stack, axis=0)
    
    for i in range(max_confidence_indices.shape[1]):  # Iterate over each prediction
        if max_confidence_indices[0, i] == happiness_index:
            happiness_score = predictions_stack[happiness_index, i]
            surprise_score = predictions_stack[surprise_index, i]
            if happiness_score * threshold_ratio >= surprise_score:
                max_confidence_indices[0, i] = surprise_index  # Reassign to 'surprise'
                change_count += 1  # Increment the count
    
    return max_confidence_indices, change_count

# Define the threshold ratio; for example, surprise should be no more than 20% less confident than happiness
threshold_ratio = 0.8  # 20% below happiness

# Stack predictions for voting
predictions_stack = np.vstack([predictions_v3, predictions_v4, predictions_v6])

# Apply the reassigning logic and get the count of changes
final_predictions, sentences_changed = reassign_classes_with_count(predictions_stack, threshold_ratio)

# Convert numeric predictions to emotion labels
final_predictions_labels = [emotion_mapping[pred] for pred in final_predictions.flatten()]

# Print the final predictions and the amount of sentences changed
print(final_predictions_labels)
print(f"Number of sentences reassigned from 'happiness' to 'surprise': {sentences_changed}")

In [ ]:
import joblib

# Load the LabelEncoder for V6 since you are using predictions based on the large model
label_encoder_path_v6 = 'Roberta_V3_1_task12/label_encoder.joblib'
label_encoder = joblib.load(label_encoder_path_v6)

# Convert numeric predictions back to original labels using the LabelEncoder
predicted_emotions = label_encoder.inverse_transform(final_predictions)

# Prepare the submission DataFrame
submission_df = pd.DataFrame({
    'id': test_df['id'],  # Assuming there's an 'id' column in your test dataset
    'emotion': predicted_emotions
})

# Save the submission file
submission_df.to_csv('Roberta_ensemble_v3.csv', index=False, sep=',')


In [ ]:
data_sub = 'Roberta_ensemble_v3.csv'

# Load the dataset
data_submission = pd.read_csv(data_sub)

# Define the mapping from integer labels to emotion names
emotion_mapping = {
    0: 'anger',
    1: 'disgust',
    2: 'fear',
    3: 'happiness',
    4: 'sadness',
    5: 'surprise'
}

# Apply the mapping to the 'emotion' column
data_submission['emotion'] = data_submission['emotion'].map(emotion_mapping)
# Save the submission file
data_submission.to_csv('Roberta_ensemble_v3.csv', index=False, sep=',')

In [ ]:
# Calculate the average confidence score per class for each model
average_confidence_v2 = get_average_confidence_score(predictions_v2)
average_confidence_v3 = get_average_confidence_score(predictions_v3)
average_confidence_v4 = get_average_confidence_score(predictions_v4)
average_confidence_v6 = get_average_confidence_score(predictions_v6)

# Now you have the average confidence per class for each model
print("Average confidence per class for model_v2:", average_confidence_v2)
print("Average confidence per class for model_v3:", average_confidence_v3)
print("Average confidence per class for model_v4:", average_confidence_v4)
print("Average confidence per class for model_v6:", average_confidence_v6)